In [1]:
import pandas as pd
import numpy as np

In [2]:
def rev_comp(strSeq):
	dicComp = {'A':'T','T':'A','G':'C','C':'G','N':'N'}
	strCseq = ''
	for i in strSeq:
		try:
			strCseq += dicComp[i.upper()]
		except KeyError:
			strCseq += 'N'
	# End of for i
	return(strCseq[::-1])

In [4]:
file_fa = '/ref/analysis/Cre/ref/Creinhardtii_281_v5.0.fa'

bulk = open(file_fa).read()
bulk = bulk.split('>')
dicHD2seq = {}
for each_bulk in bulk:
    if each_bulk.strip() == '':
        continue
    genename = each_bulk.split('\n')[0]
    seq      = ''.join(each_bulk.split('\n')[1:])
    dicHD2seq[genename] = seq

In [5]:
file_gff = '/ref/analysis/Cre/braker/braker.try5_mario/braker/chlamy2011/augustus.gff.rmsharp.gff.parsed.gff'

df_gff = pd.read_csv(file_gff,sep='\t',header=None)

mask        = (df_gff[2] == 'gene')
df_gff_gene = df_gff[mask]      

df_gff_gene['genename'] = df_gff[8].apply(lambda x:x.replace('ID=',''))

mask = (df_gff[2] == 'CDS')
df_gff_cds = df_gff[mask]      

df_gff_cds['genename'] = df_gff_cds[8].apply(lambda x : x.replace('Parent=',''))

df_gff_cds_index = df_gff_cds.set_index('genename')

df_gff_cds_index = df_gff_cds_index.sort([3], ascending=[1])

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [7]:
i = 0
dic = {'transcriptname' : [],
       'strand' : [],
       'CDSloc' : [],
       'CDSseq' : []}
for genename in set(df_gff_cds_index.index):
    
    dic['transcriptname'].append(genename)
    
    edf = df_gff_cds_index.loc[genename]
    if str(type(edf)) == "<class 'pandas.core.frame.DataFrame'>":
        CDS_list = zip(list(edf[3]),list(edf[4]))
        chromosome = edf[0][0]
        strand     = edf[6][0]
    else:
        CDS_list = zip([edf[3]],[edf[4]])
        chromosome = edf[0]
        strand     = edf[6]
    
    dic['strand'].append(strand)
    cdsseq = ''
    for l,r in CDS_list:
        #print l,r
        cdsseq += dicHD2seq[chromosome][l-1:r]
    if strand == '+':
        pass
    else: 
        cdsseq = rev_comp(cdsseq)
    dic['CDSloc'].append(CDS_list)
    dic['CDSseq'].append(cdsseq)
    #print genename,chromosome,strand,CDS_list,cdsseq
    #if i == 10:
    #    break
    #i += 1

In [8]:
df_cds = pd.DataFrame(dic)
df_cds['transcript'] = df_cds['transcriptname'].apply(lambda x : x.split('.')[1])
df_cds['gene'] = df_cds['transcriptname'].apply(lambda x : x.split('.')[0])

In [9]:
df_cds_index = df_cds.set_index(['gene','transcript'])

In [10]:
df_cds_index.to_pickle(file_gff+'.cdsseq.pk')

In [11]:
df_cds_index.head()

,,CDSloc,CDSseq,strand,transcriptname
gene,transcript,,,,
g13850,t1,"[(6430310, 6430466), (6430692, 6430891), (6431...",ATGAGAACGCTGAAGAAAATTAGGCTGGAGCAGGAGGATGAGGGCG...,-,g13850.t1
g3773,t1,"[(5367703, 5367731), (5367818, 5367988), (5368...",ATGGCAACGTTGAGGCTCGATGTCAGCGGGCGACTCGACGAGGAGG...,+,g3773.t1
g9804,t1,"[(1998351, 1998382), (1998509, 2000696), (2000...",ATGGGCCGCAACCACCGCCACGCCCGTACCCAGAACCACAGTCACG...,-,g9804.t1
g12808,t1,"[(69755, 69943), (70159, 70284), (70868, 70972)]",ATGACCCATCCCGGCGCCGCCCCGGCCGCTGCCACCCTCCCCTGTC...,+,g12808.t1
g3484,t1,"[(3884746, 3884791), (3884937, 3886198), (3886...",ATGGCGGAGCCGGTGGAGGACGTGGTCGTGCTGGACAGCCTACAGC...,+,g3484.t1
